# Wild Blueberry Yield Prediction

"The dataset used for predictive modeling was generated by the Wild Blueberry Pollination Simulation Model, which is an open-source, spatially-explicit computer simulation program that enables exploration of how various factors, including plant spatial arrangement, outcrossing and self-pollination, bee species compositions and weather conditions, in isolation and combination, affect pollination efficiency and yield of the wild blueberry agroecosystem. The simulation model has been validated by the field observation and experimental data collected in Maine USA and Canadian Maritimes during the last 30 years and now is a useful tool for hypothesis testing and theory development for wild blueberry pollination researches."

The aim it to predict blueberry yield

Features Unit Description:
- Clonesize m2 The average blueberry clone size in the field
- Honeybee bees/m2/min Honeybee density in the field
- Bumbles bees/m2/min Bumblebee density in the field
- Andrena bees/m2/min Andrena bee density in the field
- Osmia bees/m2/min Osmia bee density in the field
- MaxOfUpperTRange ℃ The highest record of the upper band daily air temperature during the bloom season
- MinOfUpperTRange ℃ The lowest record of the upper band daily air temperature
- AverageOfUpperTRange ℃ The average of the upper band daily air temperature
- MaxOfLowerTRange ℃ The highest record of the lower band daily air temperature
- MinOfLowerTRange ℃ The lowest record of the lower band daily air temperature
- AverageOfLowerTRange ℃ The average of the lower band daily air temperature
- RainingDays Day The total number of days during the bloom season, each of which has precipitation larger than zero
- AverageRainingDays Day The average of raining days of the entire bloom season
- 'fruitset', 'fruitmass', 'seeds' **non** defined

Resources:
- [Kaggle challenge](https://www.kaggle.com/competitions/playground-series-s3e14/overview)

In [ ]:
# importing standard libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve, LearningCurveDisplay, ShuffleSplit

from xgboost import XGBRegressor

import lightgbm as lgb

In [ ]:
# Define Seaborn theme parameters
theme_parameters =  {
    'axes.spines.right': False,
    'axes.spines.top': False,
    'grid.alpha':0.3,
    'axes.titlesize': 16,
    'figure.figsize': (12, 4),
}

# Set the theme
sns.set_theme(style='whitegrid',
              palette=sns.color_palette('colorblind'), 
              rc=theme_parameters)

# Read Data

In [ ]:
read_from_kaggle = True

In [ ]:
if read_from_kaggle:
    
    df_train = pd.read_csv("/kaggle/input/playground-series-s3e14/train.csv")
    df_test = pd.read_csv("/kaggle/input/playground-series-s3e14/test.csv")
    
else:

    df_train = pd.read_csv("./../data/blueberry_train.csv")
    df_test = pd.read_csv("./../data/blueberry_test.csv")

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
# drop id column
df_train = df_train.drop(columns='id', axis=1)

# Exploratory Data Analysis - EDA

## Features distribution train vs validation set KDEs

In [ ]:
figure, ax = plt.subplots(4, 4, figsize=(16, 12))
ax = ax.flatten()

for index, col_name in enumerate(df_train.columns[:-1]):
    
    sns.kdeplot(data=df_train[col_name],
                label='Train',
                ax=ax[index])
    
    sns.kdeplot(data=df_test[col_name],
                label='Test',
                ax=ax[index])
   
    ax[index].set_title(col_name, fontsize=14)
    
    ax[index].tick_params(labelrotation=45)
    
    # Retrieve legend information
    handles = ax[index].get_legend_handles_labels()[0]
    labels = ax[index].get_legend_handles_labels()[1]
    ax[index].legend().remove()

# Set the legend
figure.legend(handles, 
              labels, 
              loc='upper center', 
              bbox_to_anchor=(0.5, 1.03), 
              fontsize=12,
              ncol=3)

plt.tight_layout()

- The distribution of the features is consistend between train and validation sets

In [ ]:
figure, ax = plt.subplots(1, 2)
ax = ax.flatten()

sns.boxplot(data=df_train, 
            y='honeybee',
            ax=ax[0])

sns.boxplot(data=df_test,
            y='honeybee',
            ax=ax[1],
            color='orange')

ax[0].set_title('Train honeybee distribution')

ax[1].set_title('Test honeybee distribution')

plt.show()

In [ ]:
df_train['honeybee'].value_counts().sort_index(ascending=False).head(5)

- Outlyers of honeybees need to be dropped before modeling (just on df_train)

In [ ]:
# dropping 'honeybee' values >= 5 in test
df_train = df_train[df_train['honeybee'] < 5].reset_index()

## Label distribution KDE

In [ ]:
sns.kdeplot(data=df_train['yield'],
            label='Train')

plt.title('Label distribution (yield)')

plt.show()

The distribution of the label yield looks similar to the distribution of the features 'fruitset', 'fruitmass' and 'seeds'. Since these features are not described (see notebook intro) we can hypotesize that they have been inferred from the label yield.

## Pearson Correlation

In [ ]:
# Generate correlation matrix
correlation_train = df_train.corr(method='pearson')

In [ ]:
# Generate a mask for the upper triangle
correlation_mask = np.triu(np.ones_like(correlation_train, dtype=bool))

In [ ]:
figure, ax = plt.subplots(figsize=(14, 10))

ax = sns.heatmap(correlation_train, 
            mask=correlation_mask, 
            cmap='mako',
            vmax=1.0, 
            vmin=-1.0, 
            center=0, 
            square=True, 
            linewidths=.5, 
            annot=True,
            annot_kws={'fontsize': 8},
            cbar_kws={"shrink":.8, 'orientation':'vertical'})


ax.set_title('Correlation heatmap')

plt.show()

- The correlation matrix shows a very similar and a high correlation behaviour between the features 'fruitset', 'fruitmass' and 'seeds and the lable 'yield', thus the above hypotesis was quite likely correct
- The feature 'clonesize' and 'honeybee' looks to be positively correlated (0.85), we can hypotesize that honeybees are attracted to blueberry cultivation with higher clonesize (maybe because the domestication of honeybees). Anyway, this doesn't appear to translate into a highet yield
- The temperature measures are duplicated, it's enough to keep just one
- Same with AverageRainingDays and RainingDays. Dropping AverageRainingDays

 ## Pairplot
 
 Plotting pairplots between main features to look at potential non-linear relaationships

In [ ]:
df_train.columns

In [ ]:
sns.pairplot(df_train[['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds', 'yield']],
             kind="reg",
             diag_kind='kde',
             plot_kws={'line_kws':{'color':'red'}},
             corner=True)

plt.suptitle('Features and Target Pairplots', 
             fontsize=20, 
             fontweight='bold')

plt.show()

The data are probably AI generated. Doesn't look like there are particular non-linar relationships between features.

# Preprocessing data before modeling

## Get the datasets model-ready

In [ ]:
# dropping 'honeybee' values >= 5 in test
df_train = df_train[df_train['honeybee'] < 5].reset_index()

In [ ]:
def compute_engineered_features(data: pd.DataFrame) -> pd.DataFrame:
    
    """
    Create engineered features to have the dataset model-ready
    
    Args:
        data Pandas.DataFrame input
    
    Returns:
        data Pandas.DataFrame with engineered features
    """
    
    # Create a feature clonesize per honeybee
    data['clonesize per honeybee'] = data['clonesize'] * data['honeybee']
    
    return data

In [ ]:
# get the train data model-ready
df_train = compute_engineered_features(df_train.copy())

# get the test data model-ready
df_test = compute_engineered_features(df_test.copy())

## Defining features and label

In [ ]:
features = ['clonesize', 
            'honeybee', 
            'bumbles', 
            'andrena', 
            'osmia', 
            'AverageOfUpperTRange',
            'AverageRainingDays', 
            'fruitset', 
            'fruitmass', 
            'seeds',
            'clonesize per honeybee']

label = ['yield']

## Scaling data

In [ ]:
# Scaling train data with StandardScaler
scaler = StandardScaler(copy=True,
                        with_mean=True,
                        with_std=True)

scaler.fit(df_train[features])

df_train_scaled = pd.DataFrame(scaler.transform(df_train[features]))

# Scaling removed column names - put them back
df_train_scaled.columns = features

In [ ]:
# Scaling test data
df_test_scaled = pd.DataFrame(scaler.transform(df_test[features]))

# Scaling removed column names - put them back
df_test_scaled.columns = features

## Splitting data between train and test set

In [ ]:
# defining a seed
seed = 108

In [ ]:
# Define X and y for the training set
X = df_train_scaled
y = df_train.iloc[:, -2]

# Splitting train dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

# Model Training

In [ ]:
# Define metric(s) to compare the models
metrics = ['RMSE']

# Initialize DataFrame of model performance
performance = pd.DataFrame(columns=metrics)

## Linear Regression

In [ ]:
%%time

# Instantiate a LinearRegression model
model_lr = LinearRegression()

# Fit logreg to the train set
model_lr.fit(X_train, y_train)

# predict y_pred values
y_pred_lr = model_lr.predict(X_test)

# Compute RMSE metric
rmse_lr = round(mean_squared_error(y_test, y_pred_lr) ** 0.5, 2)

print('RMSE: {}'.format(rmse_lr))

In [ ]:
# Update 'performance' DataFrame
performance.loc['Linear Regression'] = [rmse_lr]

## Random Forest Regressor

In [ ]:
%%time

# Instantiate rf
model_rf = RandomForestRegressor(n_estimators=25,
                                 random_state=seed)
            
# Fit rf to the training set    
model_rf.fit(X_train, y_train) 

# Predict the test set labels
y_pred_rf = model_rf.predict(X_test)

# Evaluate the test set RMSE
rmse_rf = round(mean_squared_error(y_test, y_pred_rf) ** 0.5, 2)

# Print rmse_test
print('RMSE: {}'.format(rmse_rf))

In [ ]:
# Update 'performance' DataFrame
performance.loc['Random Forest Regressor'] = [rmse_rf]

## Gradient Boosting Regressor

In [ ]:
%%time

# Instantiate gb
model_gb = GradientBoostingRegressor(max_depth=4, 
                                     n_estimators=200,
                                     random_state=2)

# Fit gb to the training set
model_gb.fit(X_train, y_train)

# Predict test set labels
y_pred_gb = model_gb.predict(X_test)

# Evaluate the test set RMSE
rmse_gb = round(mean_squared_error(y_test, y_pred_gb) ** 0.5, 2)

# Print rmse_test
print('RMSE: {}'.format(rmse_gb))

In [ ]:
# Update 'performance' DataFrame
performance.loc['Gradient Boosting Regressor'] = [rmse_gb]

## XGBoost Regressor

In [ ]:
# define XGBoost hyperparams dict
hyperparams_xgb = {
    'n_estimators' : 500,
    'max_depth' : 5,
    'learning_rate' : 0.01
}

In [ ]:
# Define the model
model_xgb = XGBRegressor(**hyperparams_xgb)

# Train the xgb to the training set
model_xgb.fit(X_train, y_train)

# Get predictions
y_pred_xgb = model_xgb.predict(X_test)

# Compute metrics
rmse_xgb = round(mean_squared_error(y_test, y_pred_xgb) ** 0.5, 2)

print('RMSE: {}'.format(rmse_xgb))

In [ ]:
# Update 'performance' DataFrame
performance.loc['XGBoost Regressor'] = [rmse_xgb]

## LightGBM

In [ ]:
# define LightGBM hyperparams dict
hyperparams_lgb = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'force_row_wise': True,
    'n_estimators': 1000,
    'verbose': 0
}

In [ ]:
%%time

# Define the model
model_lgb = lgb.LGBMRegressor(**hyperparams_lgb)

# Fit model to the training set
model_lgb.fit(X_train, np.ravel(y_train))

# Get predictions
y_pred_lgb = model_lgb.predict(X_test)

# Compute metrics
rmse_lgb = round(mean_squared_error(y_test, y_pred_lgb) ** 0.5, 2)

print('RMSE: {}'.format(rmse_lgb))

In [ ]:
# Update 'performance' DataFrame
performance.loc['LightGBM Regressor'] = [rmse_lgb]

# Model comparison

In [ ]:
# Sort dataframe by the metric
performance.sort_values('RMSE', inplace=True)

# Plot model metrics
ax = sns.barplot(data=performance, 
                 x='RMSE', 
                 y=performance.index.tolist())

ax.set_title('Model comparison')

plt.show()

## Tuning best sklearn model hyperparameters

In [ ]:
%%time

# Define a dictionary containing the parameters to tune
params_model_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
}

# Instantiate grid_gb
grid_model_gb = GridSearchCV(estimator=GradientBoostingRegressor(random_state=seed),
                       param_grid=params_model_gb,
                       scoring='neg_mean_squared_error',
                       cv=3,
                       verbose=1,
                       n_jobs=-1)

# Fit grid_model_rf
grid_model_gb.fit(X_train, y_train)

In [ ]:
# Extract best hyperparams
best_params_gb = grid_model_gb.best_params_

print('Best hyperparameters for Gradient Boosting Regressor:\n', best_params_gb)

In [ ]:
# Extract the best model performance
best_model_gb = grid_model_gb.best_estimator_

# Fit best_model_gb to the training set
best_model_gb.fit(X_train, y_train)

# Predict test set labels using the best model
y_pred_best_gb = best_model_gb.predict(X_test)

# Evaluate the test set RMSE
rmse_best_gb = round(mean_squared_error(y_test, y_pred_best_gb) ** 0.5, 2)

# Print rmse_test
print('RMSE best gb: {}'.format(rmse_best_gb))

In [ ]:
# Update 'performance' DataFrame
performance.loc['Tuned Gradient Boosting Regressor'] = [rmse_best_gb]

## Tuning XGBoost model

In [ ]:
%%time

# Define a dictionary containing the parameters to tune
params_model_xgb = {
    'n_estimators': [55, 250, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
}

# Instantiate grid_gb
grid_model_xgb = GridSearchCV(estimator=XGBRegressor(random_state=seed),
                       param_grid=params_model_xgb,
                       scoring='neg_mean_squared_error',
                       cv=3,
                       verbose=1,
                       n_jobs=-1)

# Fit grid_model_rf
grid_model_xgb.fit(X_train, y_train)

In [ ]:
# Extract best hyperparams
best_params_xgb = grid_model_xgb.best_params_

print('Best hyperparameters for XGBoost model:\n', best_params_xgb)

In [ ]:
# Extract the best model performance
best_model_xgb = grid_model_xgb.best_estimator_

# Fit best_model_gb to the training set
best_model_xgb.fit(X_train, y_train)

# Predict test set labels using the best model
y_pred_best_xgb = best_model_xgb.predict(X_test)

# Evaluate the test set RMSE
rmse_best_xgb = round(mean_squared_error(y_test, y_pred_best_xgb) ** 0.5, 2)

# Print rmse_test
print('RMSE best xgb: {}'.format(rmse_best_xgb))

In [ ]:
# Update 'performance' DataFrame
performance.loc['Tuned XGBoost Regressor'] = [rmse_best_xgb]

## Updating model comparison after tuning the better performing models

In [ ]:
# Sort dataframe by the metric
performance.sort_values('RMSE', inplace=True)

# Plot model metrics
ax = sns.barplot(data=performance, 
                 x='RMSE', 
                 y=performance.index.tolist())

ax.set_title('Model comparison updated')

plt.show()

# Model Diagnostic

Diagnostic the fit of the best performing models
 - (tuned) xgboost regressor
 - sklearn (tuned) gradient boosting regressor

## Feature Importance

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

# Compute the feature importance
importances_xgb = sorted(list(zip(best_model_xgb.feature_names_in_,
                              best_model_xgb.feature_importances_)))

# Transform it into a DataFrame
importances_xgb_df = pd.DataFrame(importances_xgb,
                                     columns= ['Feature', 'Importance'])

# Sorting importances
importances_xgb_df = importances_xgb_df.sort_values(['Importance'], ascending=False).reset_index(drop=True)

# Plot the feature importance
ax[0] = sns.barplot(data=importances_xgb_df, 
                 x='Feature', 
                 y='Importance',
                 ax=ax[0])
ax[0].set_title('XGBoost Regressor Feature Importance')
ax[0].tick_params(labelrotation=90)

# Compute the feature importance
importances_gb = sorted(list(zip(best_model_gb.feature_names_in_,
                              best_model_gb.feature_importances_)))

# Transform it into a DataFrame
importances_gb_df = pd.DataFrame(importances_gb,
                                     columns= ['Feature', 'Importance'])

# Sorting importances
importances_gb_df = importances_gb_df.sort_values(['Importance'], ascending=False).reset_index(drop=True)

# Plot the feature importance
ax[1] = sns.barplot(data=importances_gb_df, 
                 x='Feature', 
                 y='Importance',
                 ax=ax[1])
ax[1].set_title('Gradient Boosting Regressor Feature Importance')
ax[1].tick_params(labelrotation=90)

plt.show()

## Learning curve

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(16,4))

common_params = {
    "X": X,
    "y": y,
    "train_sizes": np.linspace(0.1, 1.0, 10),
    "cv": ShuffleSplit(n_splits=10, test_size=0.2, random_state=0),
    "score_type": "both",
    "n_jobs": 4,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "Score",
}

for ax_idx, estimator in enumerate([model_xgb, best_model_gb]):
    
    LearningCurveDisplay.from_estimator(estimator, **common_params, ax=ax[ax_idx])
    
    handles, label = ax[ax_idx].get_legend_handles_labels()
    
    ax[ax_idx].legend(handles[:2], ["Training Score", "Test Score"])
    
    ax[ax_idx].set_title(f"Learning Curve for {estimator.__class__.__name__}")
    
plt.tight_layout()

## Residuals

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(14,4))

# Create a DataFrame of residuals for best_model_xgb
residuals_xgb = y_test - y_pred_best_xgb
df_resid_xgb = pd.DataFrame({'residuals': residuals_xgb, 'y_pred_best_xgb': y_pred_best_xgb})

# Plot the residuals for best_model_xgb
sns.residplot(x='y_pred_best_xgb',
              y='residuals', 
              data=df_resid_xgb, 
              ax=ax[0])
ax[0].set_title('XGBoost Regressor Residual Plot', fontsize=16)
ax[0].set_xlabel('Predicted values')
ax[0].set_ylabel('')

# Create a DataFrame of residuals for best_model_gb
residuals_bg = y_test - y_pred_best_gb
df_resid_bg = pd.DataFrame({'residuals': residuals_bg, 'y_pred_best_gb': y_pred_best_gb})

# Plot the residuals for best_model_gb
sns.residplot(x='y_pred_best_gb', 
              y='residuals', 
              data=df_resid_bg, 
              ax=ax[1])
ax[1].set_title('Gradient Boosting Regressor Residual Plot', fontsize=16)
ax[1].set_xlabel('Predicted values')
ax[1].set_ylabel('Residuals')

plt.show()

## Q-Q Plot

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

# Calculate the residuals for best_model_xgb
residuals_xgb = y_test - y_pred_best_xgb

# Generate a QQ plot for best_model_xgb
stats.probplot(residuals_xgb, 
               plot=ax[0])
ax[0].set_title('Q-Q Plot of XGBoost Regressor Residuals', fontsize=16)
ax[0].set_xlabel('Theoretical quantiles')
ax[0].set_ylabel('')

# Calculate the residuals for best_model_gb
residuals_gb = y_test - y_pred_best_gb

# Generate a QQ plot for model 1
stats.probplot(residuals_gb, 
               plot=ax[1])
ax[1].set_title('Q-Q Plot of Gradient Boosting Regressor Residuals', fontsize=16)
ax[1].set_xlabel('Theoretical quantiles')
ax[1].set_ylabel('Sample quantiles')

plt.show()

## Actual Values vs Predicted Values

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

# Plot the predicted vs actual values for each model
ax[0].scatter(y_test, 
            y_pred_best_gb, 
            label='GB - RMSE: {}'.format(rmse_best_gb))
ax[0].plot([y_test.min(), y_test.max()], 
         [y_test.min(), y_test.max()], 
         'k--', 
         lw=2)

ax[1].scatter(y_test, 
            y_pred_best_xgb, 
            label='XGB - RMSE: {}'.format(rmse_best_xgb))
ax[1].plot([y_test.min(), y_test.max()], 
         [y_test.min(), y_test.max()], 
         'k--', 
         lw=2)

# Add a legend to the plot
ax[0].legend()
ax[1].legend()

# Set x and y axis labels for the plot
ax[0].set_xlabel('Actual values')
ax[0].set_ylabel('Predicted values')
ax[1].set_xlabel('Actual values')
ax[1].set_ylabel('Predicted values')

fig.suptitle('Predicted vs Actual Values')

plt.show()

## Diagnostic considerations

from the various diagnostic plot we see that the models behave similarly.

- They both slightly overfit the trainig set (disproportion among features contribution and steeper learning curve for the training set against the test test)
- Both models are have quite poor performance in predicting extreme values (see Q-Q plots and Predicted vs Actual values plots)

# Challenge Submission

In [ ]:
# Generate predictions with test set
submission_predictions = best_model_xgb.predict(df_test_scaled)

In [ ]:
df_test_scaled.head()

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame({'id': df_test.id, 'yield': submission_predictions})

In [ ]:
submission.head()

In [ ]:
# Write CSV File
submission.to_csv('./submission.csv', index=False)